In [1]:
%tensorflow_version 1.x
import tensorflow as tf
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten,Input, Lambda, Activation, Concatenate
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras import layers
import numpy as np
import os
import cv2
import numpy as np
import random
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
# from tensorflow.keras.preprocessing.image import image_dataset_from_directory
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# import pickle
from google.colab import files

TensorFlow 1.x selected.


Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
grayscale_training_data = np.load('/content/drive/My Drive/grayscale_training_data.npy')  
thermal_training_data = np.load('/content/drive/My Drive/thermal_training_data.npy') 

In [0]:
# to eliminate non overlapped image pairs
grayscale_training_data = np.delete(grayscale_training_data,np.s_[5000:8186],0)
thermal_training_data = np.delete(thermal_training_data,np.s_[5000:8186],0)

In [0]:
grayscale_test_data = np.load('/content/drive/My Drive/grayscale_test_data.npy')  
thermal_test_data = np.load('/content/drive/My Drive/thermal_test_data.npy') 

In [0]:
# SSIM LOSS
import tensorflow as tf
import numpy as np
from scipy.stats import entropy
lambda1 = 0
lambda2 = 1
lambda3 = 1
lambda4 = 1
def _tf_fspecial_gauss(size, sigma):
    """Function to mimic the 'fspecial' gaussian MATLAB function
    """
    x_data, y_data = np.mgrid[-size//2 + 1:size//2 + 1, -size//2 + 1:size//2 + 1]

    x_data = np.expand_dims(x_data, axis=-1)
    x_data = np.expand_dims(x_data, axis=-1)

    y_data = np.expand_dims(y_data, axis=-1)
    y_data = np.expand_dims(y_data, axis=-1)

    x = tf.constant(x_data, dtype=tf.float32)
    y = tf.constant(y_data, dtype=tf.float32)

    g = tf.exp(-((x**2 + y**2)/(2.0*sigma**2)))
    return g / tf.reduce_sum(g)


def SSIM_(img1, img2, size=11, sigma=1.5):
    window = _tf_fspecial_gauss(size, sigma) # window shape [size, size]
    K1 = 0.01
    K2 = 0.03
    L = 1  # depth of image (255 in case the image has a differnt scale)
    C1 = (K1*L)**2
    C2 = (K2*L)**2
    mu1 = tf.nn.conv2d(img1, window, strides=[1,1,1,1], padding='VALID')
    mu2 = tf.nn.conv2d(img2, window, strides=[1,1,1,1],padding='VALID')
    mu1_sq = mu1*mu1
    mu2_sq = mu2*mu2
    mu1_mu2 = mu1*mu2
    sigma1_sq = tf.nn.conv2d(img1*img1, window, strides=[1,1,1,1],padding='VALID') - mu1_sq
    sigma2_sq = tf.nn.conv2d(img2*img2, window, strides=[1,1,1,1],padding='VALID') - mu2_sq
    sigma12 = tf.nn.conv2d(img1*img2, window, strides=[1,1,1,1],padding='VALID') - mu1_mu2

    value = (2.0*sigma12 + C2)/(sigma1_sq + sigma2_sq + C2)
    value = tf.reduce_mean(value)
    return value

def SSIM_LOSS(img1, img2, size=11, sigma=1.5):
    window = _tf_fspecial_gauss(size, sigma) # window shape [size, size]
    K1 = 0.01
    K2 = 0.03
    L = 1  # depth of image (255 in case the image has a differnt scale)
    C1 = (K1*L)**2
    C2 = (K2*L)**2
    mu1 = tf.nn.conv2d(img1, window, strides=[1,1,1,1], padding='VALID')
    mu2 = tf.nn.conv2d(img2, window, strides=[1,1,1,1],padding='VALID')
    mu1_sq = mu1*mu1
    mu2_sq = mu2*mu2
    mu1_mu2 = mu1*mu2
    sigma1_sq = tf.nn.conv2d(img1*img1, window, strides=[1,1,1,1],padding='VALID') - mu1_sq
    sigma2_sq = tf.nn.conv2d(img2*img2, window, strides=[1,1,1,1],padding='VALID') - mu2_sq
    sigma12 = tf.nn.conv2d(img1*img2, window, strides=[1,1,1,1],padding='VALID') - mu1_mu2

    value = (2.0*sigma12 + C2)/(sigma1_sq + sigma2_sq + C2)
    value = tf.reduce_mean(value)
    return 1-value
def correlation_coefficient_loss(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x)
    my = K.mean(y)
    xm, ym = x-mx, y-my
    r_num = K.sum(tf.multiply(xm,ym))
    r_den = K.sqrt(tf.multiply(K.sum(K.square(xm)), K.sum(K.square(ym))))
    r = r_num / r_den

    r = K.maximum(K.minimum(r, 1.0), -1.0)
    return 1 - K.square(r)
def loss_funct(in1,out1):
  return (lambda1*loss_SSIM(in1,out1) + lambda2*(tf.norm(in1-out1, ord='euclidean')) + lambda3*correlation_coefficient_loss(in1,out1) +\
          lambda4*loss_MS_SSIM(in1, out1))

def loss_MS_SSIM(in1,out1):
  return 1-tf.image.ssim_multiscale(in1,out1,max_val=1)
def MS_SSIM(in1,out1):
  return tf.image.ssim_multiscale(in1,out1,max_val=1)

def loss_SSIM(in1,out1):
  return 1-tf.image.ssim(in1,out1,max_val=1)
def tf_SSIM(in1,out1):
  return tf.image.ssim(in1,out1,max_val=1)

def SCD(img1, fus):
    value = corr2(fus, img1)
    tf.cast(value, tf.float32)
    return value

def corr2(a,b):
    a = a - K.sum(a)
    b = b - K.sum(b)
    r = K.sum(tf.multiply(a,b)) / K.sqrt(tf.multiply(K.sum(K.square(a)) , K.sqrt(K.sum(K.square(b)))));
    return r


In [8]:
# flir dataset
def summation(vects):
    x, y = vects
    return x+y
def summation_shape(shapes):
    shape1, shape2 = shapes
    return (shape1)
def create_base_network(input_shape):
    '''Base network to b3e shared (eq. to feature extraction).
    '''
    input = Input(shape=input_shape)
    x = Conv2D(16, (3, 3),padding='same',kernel_initializer="glorot_normal",activation='relu')(input)
    x = Conv2D(32, (3, 3),padding='same',kernel_initializer="glorot_normal",activation='relu')(x)
    # x = Conv2D(64, (3, 3),padding='same',activation='relu')(x)
    return Model(input, x)

def decoder(input_shape):
    input = Input(shape=input_shape)
    # x = Conv2D(128, (3, 3),padding='same',activation='relu')(input)
    # x = Conv2D(64, (3, 3),padding='same',activation='relu')(input)
    x = Conv2D(32, (3, 3),padding='same',kernel_initializer="glorot_normal",activation='relu')(input)
    x = Conv2D(16, (3, 3),padding='same',kernel_initializer="glorot_normal",activation='relu')(x)
    out = Conv2D(1, (3, 3),padding='same',kernel_initializer="glorot_normal",activation='relu')(x)
    return Model(input, out)

input_shape = thermal_training_data.shape[1:]
base_network = create_base_network(input_shape)


input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)
processed_a = base_network(input_a)
processed_b = base_network(input_b)

sum_shape = thermal_training_data.shape[1:]
aa = Lambda(summation,
                  output_shape=sum_shape)([processed_a, processed_b])
# aa = Concatenate()([processed_a, processed_b])

base_decoder_network = decoder((256,256,32))
out1 = base_decoder_network(aa)
out2 = base_decoder_network(aa)


model = Model(inputs=[input_a, input_b], outputs=[out1,out2])       


opt = keras.optimizers.Adam(lr=0.001)
model.compile(loss=loss_funct,optimizer=opt,metrics=[loss_funct,MS_SSIM,tf_SSIM])
model.summary()

history = model.fit([grayscale_training_data,thermal_training_data], [grayscale_training_data,thermal_training_data],
                epochs=100,
                batch_size=128,
                shuffle=True,
                # validation_data=([grayscale_training_data,thermal_training_data], [grayscale_training_data,thermal_training_data]))
                validation_data=([grayscale_test_data,thermal_test_data], [grayscale_test_data,thermal_test_data]))

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
model_4 (Model)                 (None, 256, 256, 32) 4800        input_6[0][0]                    
                                                                 input_7[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 256, 256, 1)  0           model_4[1][0]              

In [0]:
# save model
model.save('v2_flir_lamba1_0_lamba2_1_lamba3_1_lamba4_1.h5')
files.download('v2_flir_lamba1_0_lamba2_1_lamba3_1_lamba4_1.h5')

In [0]:
aa = (history.history['val_'+model.layers[4].name+'_tf_SSIM_1'])   
bb = (history.history['val_'+model.layers[4].name+'_tf_SSIM'])
np.save("val_tf_SSIM_1", aa)
np.save("val_tf_SSIM", bb)
files.download('val_tf_SSIM_1.npy') 
files.download('val_tf_SSIM.npy') 

cc = (history.history['val_'+model.layers[4].name+'_MS_SSIM_1'])   
dd = (history.history['val_'+model.layers[4].name+'_MS_SSIM'])
np.save("val_MS_SSIM_1", cc)
np.save("val_MS_SSIM", dd)
files.download('val_MS_SSIM_1.npy') 
files.download('val_MS_SSIM.npy') 

ee = (history.history['val_'+model.layers[4].name+'_loss_funct_1'])   
ff = (history.history['val_'+model.layers[4].name+'_loss_funct'])
np.save("val_loss_funct_1", ee)
np.save("val_loss_funct", ff)
files.download('val_loss_funct_1.npy') 
files.download('val_loss_funct.npy') 

aaa = (history.history[model.layers[4].name+'_tf_SSIM_1'])   
bbb = (history.history[model.layers[4].name+'_tf_SSIM'])
np.save("_tf_SSIM_1", aaa)
np.save("_tf_SSIM", bbb)
files.download('_tf_SSIM_1.npy') 
files.download('_tf_SSIM.npy') 

ccc = (history.history[model.layers[4].name+'_MS_SSIM_1'])   
ddd = (history.history[model.layers[4].name+'_MS_SSIM'])
np.save("_MS_SSIM_1", ccc)
np.save("_MS_SSIM", ddd)
files.download('_MS_SSIM_1.npy') 
files.download('_MS_SSIM.npy') 

eee = (history.history[model.layers[4].name+'_loss_funct_1'])   
fff = (history.history[model.layers[4].name+'_loss_funct'])
np.save("_loss_funct_1", eee)
np.save("_loss_funct", fff)
files.download('_loss_funct_1.npy') 
files.download('_loss_funct.npy') 

